# Uploading our stuff, but without 10% loss

    First, lets import our tools & ingredients again

In [1]:
import pandas as pd
import psycopg2

In [2]:
df = pd.read_excel('map.xlsx')
print(df.head())

   Ser                  combined
0    1               Agriculture
1    2    Agricultural Equipment
2    3  Agricultural Greenhouses
3    4     Aquaculture Equipment
4    5          Aquaculture Trap


In [3]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

In [5]:
def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return name, num

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; j = 0
    
    for i in range(start, end):
        name, num = get_category_subcategory(df, i); num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name))
        except psycopg2.errors.UniqueViolation as e:
            listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
        
        if(j==10):
            con.commit(); j = 0 # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions)); return listOfCollisions

In [ ]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

for i in range(46): # first 46 hundreds - should do it, but 15 mins seems like an issue - but lets buckle up none the less
    collisions = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    print("100 number ", (i+1) ," done")

In [6]:
cursor.close()
con.close()

    Now, lets run the first couple hundreds from the O(n) method, but lets make 1 key change

In [7]:
def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']                                                                             # O( log(n) )
    name = df.loc[index, 'combined']
    return name, num

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; listOfAllErrors = []; j = 0
    
    for i in range(start, end):                                                                                 # O(n)
        name, num = get_category_subcategory(df, i); num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name)) # O(n)
        except psycopg2.errors.UniqueViolation as e:
            listOfAllErrors.append(i); listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
            listOfAllErrors.append(i)
        
        if(j==10):
            con.commit(); j = 0 # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions), len(listOfAllErrors)); return listOfCollisions, listOfAllErrors

    # this should give us some details on where our data is going

In [ ]:
listOfCollisions = []; listOfAllErrors = []

# wipe the db from DBeaver
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432"); cursor = con.cursor() # open

for i in range(5): # first 5 hundreds for sake of inspection
    collisions, errors = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    listOfCollisions.append(collisions); listOfAllErrors.append(errors)
    print("100 number ", (i+1) ," done")

cursor.close(); con.close() # close

# map, batch, batching
# more then 1 value at a time

    Upon realizing there is a better way to do this, its time to inspect the database for this instead

    Lets inspect the dataset in DBeaver

![DBeaver pic](DBeaverShot.PNG "Screenshot from DBeaver")

    Looks like SER 142 through to 151 don't exist, I SHALL INSPECT

In [8]:
print(df.iloc[135:155]) # I thought the issue was in the dataset, this piece of code was to inspect

     Ser                              combined
135  136                              Palm Oil
136  137                          Rapeseed Oil
137  138                            Sesame Oil
138  139                           Soybean Oil
139  140                         Sunflower Oil
140  141                         Plant Extract
141  142                           Plant Fiber
142  143                   Plant Seeds & Bulbs
143  144       Flower Bulbs, Seeds & Seedlings
144  145                          Forage Seeds
145  146  Fruit Grafts, Seedlings & Rootstocks
146  147                             Oil Seeds
147  148                       Vegetable Seeds
148  149                               Tobacco
149  150                                   NaN
150  151                                   NaN
151  152                               Apparel
152  153               Apparel Design Services
153  154           Apparel Processing Services
154  155                         Apparel Stock


In [10]:
print(df.iloc[125:130])

     Ser            combined
125  126  Plant & Animal Oil
126  127          Animal Oil
127  128         Blended Oil
128  129        Camellia Oil
129  130          Castor Oil


    so it isn't the ampersand thats messing

In [17]:
print(df.iloc[10:30])

    Ser                    combined
10   11  Agricultural Growing Media
11   12                        Peat
12   13  Agricultural Product Stock
13   14          Agricultural Waste
14   15              Animal Extract
15   16                 Animal Feed
16   17             Animal Products
17   18                        Eggs
18   19            Fowl & Livestock
19   20        Bamboo Raw Materials
20   21             Bean-Like Seeds
21   22                       Cacao
22   23                Coffee Beans
23   24               Vanilla Beans
24   25                       Beans
25   26                 Broad Beans
26   27                Butter Beans
27   28                   Chickpeas
28   29                Kidney Beans
29   30                     Lentils


    Doesn't look like any pattern i've seen

In [18]:
print(df.iloc[25:35])

    Ser      combined
25   26   Broad Beans
26   27  Butter Beans
27   28     Chickpeas
28   29  Kidney Beans
29   30       Lentils
30   31    Lima Beans
31   32   Other Beans
32   33          Peas
33   34      Soybeans
34   35   Vigna Beans


    It definitely isn't the length of our strings

In [19]:
print(df.iloc[255:265])

     Ser               combined
255  256       Pants & Trousers
256  257        Sewing Supplies
257  258  Other Sewing Supplies
258  259         Sewing Needles
259  260         Sewing Threads
260  261     Tailor\'s Scissors
261  262                 Shorts
262  263                 Skirts
263  264              Sleepwear
264  265               Babydoll


    It also doesn't seem the backslash or apostrophe prevented it

~~It seems like I will resign to the 90.33% of the data for now, however if this disrupts the tree construction significantly I will return to the experiment from earlier with a new debug lens~~

    Time to run the experiment from earlier

In [8]:
listOfCollisions = []; listOfAllErrors = []

# wipe the db from DBeaver
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432"); cursor = con.cursor() # open

for i in range(5): # first 5 hundreds for sake of inspection
    collisions, errors = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    listOfCollisions += collisions; listOfAllErrors += errors
    print("100 number ", (i+1) ," done")

cursor.close(); con.close() # close

# map, batch, batching
# more then 1 value at a time

0 0
100 number  1  done
1 1
100 number  2  done
0 0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

1 2
100 number  4  done
0 0
100 number  5  done


In [9]:
print("errors ", listOfAllErrors);      print("; collisions ", listOfCollisions)

errors  [150, 339, 340]
; collisions  [150, 339]


In [14]:
print(df.iloc[150:151]); print(df.iloc[339:341])

     Ser combined
150  151      NaN
     Ser                    combined
339  340                         NaN
340  341  Automobiles & Motorcycle s


    Interesting, lets check DBeaver

![DBeaver pic 2](DBeaverShotEndIndex.PNG "Screenshot from DBeaver")

    looks like SER 500 was 480th

    Lets run that again for more inspection

In [15]:
listOfCollisions = []; listOfAllErrors = []

# wipe the db from DBeaver
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432"); cursor = con.cursor() # open

for i in range(5): # first 5 hundreds for sake of inspection
    collisions, errors = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    listOfCollisions += collisions; listOfAllErrors += errors
    print("100 number ", (i+1) ," done")

cursor.close(); con.close() # close

0 0
100 number  1  done
1 1
100 number  2  done
0 0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

1 2
100 number  4  done
0 0
100 number  5  done


|collisions|errors|                                                  |
|--------|----------|--------------------------------------------------|
| 0      | 0        |                                                  |
| 100    | number   | 1  done                                          |
| 1      | 1        |                                                  |
| 100    | number   | 2  done                                          |
| 0      | 0        |                                                  |
| 100    | number   | 3  done                                          |
|        |          | Error occurred: current transaction is aborted, commands ignored until end of transaction block |
| 1      | 2        |                                                  |
| 100    | number   | 4  done                                          |
| 0      | 0        |                                                  |
| 100    | number   | 5  done                                          |

    Same output, our code looks deterministic


In [16]:
print("errors ", listOfAllErrors);      print("; collisions ", listOfCollisions)

errors  [150, 339, 340]
; collisions  [150, 339]


In [17]:
print(df.iloc[150:151]); print(df.iloc[339:341])

     Ser combined
150  151      NaN
     Ser                    combined
339  340                         NaN
340  341  Automobiles & Motorcycle s


![DBeaver pic](DBeaverShotSame.PNG "Screenshot from DBeaver")

    & this looks the same too - so the issue cant be coming from our commit patterns or anything

    One calculation that should be done however, is find the number of NaN's that exist here
    For instance: 150 & 339 were NaN & appeared in both the collisions & errors list,
    But 340 which disappeared for unknown reasons was only in errors
    This is important because there is nothing we can do with a NaN, but we can do plenty with a string not transposing

In [18]:
listOfCollisions = []; listOfAllErrors = []

# wipe the db from DBeaver
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432"); cursor = con.cursor() # open

for i in range(46): # all 4600 for the sake of analysis
    collisions, errors = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    listOfCollisions += collisions; listOfAllErrors += errors
    print("100 number ", (i+1) ," done")

cursor.close(); con.close() # close

0 0
100 number  1  done
1 1
100 number  2  done
0 0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

1 2
100 number  4  done
0 0
100 number  5  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

1 8
100 number  6  done
0 0
100 number  7  done
Error occurred: current transaction is aborted, commands

KeyError: 4596

In [19]:
print("errors ", listOfAllErrors);      print("; collisions ", listOfCollisions)

errors  [150, 339, 340, 533, 534, 535, 536, 537, 538, 539, 540, 747, 748, 749, 750, 925, 926, 927, 928, 929, 930, 1009, 1010, 1068, 1069, 1070, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1465, 1466, 1467, 1468, 1469, 1470, 1509, 1510, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1558, 1559, 1560, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1948, 1949, 1950, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2015, 2016, 2017, 2018, 2019, 2020, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2577, 2578, 2579, 2580, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 3036, 3037, 3038, 3039, 3040, 3118, 3119, 3120, 3204, 3205, 3206, 3207, 3208, 3209, 3210

In [23]:
print("num errors ", len(listOfAllErrors));      print("num collisions ", len(listOfCollisions))

num errors  258
num collisions  44


    In other words

In [29]:
print("Number of missing entries in DBeaver             : ", (4591 - 4147))
print("Number of entries not going through on this side : ", len(listOfAllErrors))
print("Number of NaN entries not going through          : ", len(listOfCollisions))

Number of missing entries in DBeaver             :  444
Number of entries not going through on this side :  258
Number of NaN entries not going through          :  44


    Unfortunately this looks like multiple sources of error, in an attempt to keep the timeline moving I will obtain 1 last insight before accepting this error & attempting the DB tree in SQL